In [1]:
import pandas as pd
aircraft=pd.read_excel('.\\aircraft.xlsx')
metaData = pd.read_excel('.\\eADMSPUB_DataDictionary.xlsx')

In [2]:
findings=pd.read_excel('.\\Findings.xlsx')
events=pd.read_excel('.\\events.xlsx')
events_sequence=pd.read_excel('.\Events_Sequence.xlsx')
narratives=pd.read_excel('.\\narratives.xlsx')
ev_ID=aircraft['ev_id'].to_list()
ev_ID=list(set(ev_ID))

In [5]:
aircraft=aircraft[(aircraft['far_part']=='121')|(aircraft['far_part']=='091')]
aircraft=aircraft[aircraft['acft_category']=='AIR']
aircraft

,ev_id,Aircraft_Key,regis_no,ntsb_no,acft_missing,far_part,flt_plan_filed,flight_plan_activated,damage,acft_fire,...,acft_year,fuel_on_board,commercial_space_flight,unmanned,ifr_equipped_cert,elt_mounted_aircraft,elt_connected_antenna,elt_manufacturer,elt_model,elt_reason_other
0,20080107X00026,1,N8037W,SEA08LA057A,N,091,NONE,NaN,MINR,NONE,...,NaN,NaN,False,False,False,False,False,NaN,NaN,NaN
1,20080107X00026,2,N15EX,SEA08LA057B,N,091,NONE,NaN,SUBS,NONE,...,NaN,NaN,False,False,False,False,False,NaN,NaN,NaN
2,20080107X00027,1,N398J,DFW08LA055,N,091,IFR,Y,NaN,NONE,...,NaN,NaN,False,False,False,False,False,NaN,NaN,NaN
3,20080109X00036,1,N579RM,DFW08CA054,N,091,NONE,N,SUBS,NONE,...,NaN,NaN,False,False,False,False,False,NaN,NaN,NaN
6,20080114X00044,1,N109ND,MIA08LA036,N,091,NONE,N,SUBS,NONE,...,NaN,NaN,False,False,False,False,False,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28356,20240920195155,1,UNREG,CEN24LA361,N,091,NONE,N,SUBS,NONE,...,NaN,NaN,False,False,False,False,False,NaN,NaN,NaN
28370,20240924195177,1,N5407B,ANC24LA096,N,091,NONE,N,SUBS,NONE,...,NaN,45.0,False,False,False,True,True,Artex,Artex 345,NaN
28371,20240924195178,1,N691E,ANC24LA097,N,091,NONE,N,SUBS,NONE,...,NaN,10.8,False,False,False,True,True,NaN,NaN,NaN
28375,20240924195182,1,N303T,ANC24LA101,N,091,NONE,N,SUBS,NONE,...,1953.0,20.0,False,False,False,True,True,Kannad,Compact,NaN


In [6]:
findings=findings[(findings['Cause_Factor']=='F')|(findings['Cause_Factor']=='C')]

In [7]:
df=pd.DataFrame(ev_ID,columns=['ev_id'])
merged1=pd.merge(df,events[['ev_id','ev_date']],on='ev_id',how='left')
df_narratives=narratives[['ev_id','narr_accf','narr_cause']]
merged2=pd.merge(merged1,df_narratives,on='ev_id',how='left')

In [8]:
df_dropna=merged2.dropna(subset=['narr_accf','narr_cause'])

In [9]:
import re
def deriveNamebyCode(code):
    try:
        label = metaData[metaData['code_iaids'] == code]['meaning'].values[0]
    except:
        return 'Unknown quantity'
    
    return label

In [10]:
def get_occurence(text):
    event=str(text)
    if len(event)==2:
        code='xxx0'+event
    else:
        code='xxx'+event
    return deriveNamebyCode(code)

In [13]:
events_sequence['occurence']=events_sequence['eventsoe_no'].map(get_occurence)

In [14]:
events_drop=events_sequence[events_sequence['occurence']!='Unknown quantity']

In [15]:
grouped = events_drop.groupby('ev_id')['occurence'].agg(set).reset_index()

In [16]:
df1=pd.merge(df_dropna,grouped,on='ev_id',how='left')

In [17]:
def find_s(des,i):
    category=des.split('-')
    if i==1:
        return category[0]
    if i==2 and len(category)>=2:
        return category[1]
    if i==3 and len(category)>=3:
        return category[2]
    if i==4 and len(category)>=4:
        return category[3]


In [18]:
findings['category']=findings['finding_description'].apply(find_s,i=1)
findings_drop=findings[findings['category']!='main system']
findings_drop['category'].value_counts()
findings_grouped=findings_drop.groupby('ev_id')['category'].agg(set).reset_index()
findings_grouped

,ev_id,category
0,20080107X00026,{Personnel issues}
1,20080107X00027,"{Environmental issues, Personnel issues}"
2,20080109X00036,"{Aircraft, Environmental issues}"
3,20080114X00044,{Personnel issues}
4,20080115X00046,"{Aircraft, Personnel issues}"
...,...,...
17286,20200915X34450,{Aircraft}
17287,20200915X70606,"{Aircraft, Personnel issues}"
17288,20200917X42444,{Personnel issues}
17289,20200920X32151,"{Aircraft, Personnel issues}"


In [19]:
df2=pd.merge(df1,findings_grouped,on='ev_id',how='left')
df3=df2.dropna(subset=['category'])
df3

,ev_id,ev_date,narr_accf,narr_cause,occurence,category
3,20180301X55111,2018-02-28,"The flight instructor reported that, after con...",The pilot receiving instruction’s and flight i...,{Control flight into terr/obj},"{Aircraft, Personnel issues}"
4,20131030X22505,2013-10-30,"During the flight, the flight instructor moved...",The pilot receiving instruction’s failure to m...,{Miscellaneous/other},"{Aircraft, Personnel issues}"
5,20150626X00651,2015-06-26,A witness reported seeing the pilot depart the...,The pilot’s incapacitation in flight due to a ...,"{Loss of control in flight, Collision with ter...",{Personnel issues}
6,20160516X83754,2016-05-14,The commercial pilot indicated that he purchas...,The pilot's failure to maintain airplane contr...,"{Loss of control in flight, Collision with ter...","{Environmental issues, Personnel issues}"
7,20140522X72737,2014-05-21,The solo pilot was doing stop and go landings ...,The pilot's failure to maintain directional co...,"{Nose over/nose down, Loss of control on ground}","{Aircraft, Personnel issues}"
...,...,...,...,...,...,...
22509,20130826X54510,2013-08-24,The pilot reported that while taxiing for take...,The pilot’s decision to depart with a known de...,"{Off-field or emergency landing, Landing gear ...","{Aircraft, Personnel issues}"
22511,20120611X12331,2012-06-07,The airplane was departing the local airstrip ...,The partial loss of engine power for reasons t...,"{Off-field or emergency landing, Loss of engin...",{Not determined}
22515,20110623X12016,2011-06-20,"According to the pilot, during the turn from t...",A partial loss of engine power due to the pilo...,"{Collision with terr/obj (non-CFIT), Aircraft ...","{Aircraft, Personnel issues}"
22516,20110624X35045,2011-06-23,The pilot reported that he was landing on a be...,The pilot's improper use of brakes during land...,{Nose over/nose down},"{Aircraft, Personnel issues}"


In [20]:
df3.to_excel('NTSBdata.xlsx',index=None)